In [27]:
import geopandas as gpd
import pandas as pd
import numpy as np
import os
wijken = None
# load all parquet files into a single geodataframe from results/filter2
for file in os.listdir('results/filter2/wijken'):
    if file.endswith('.parquet'):
        if 'wijken' in locals():
            wijken = pd.concat([wijken, gpd.read_parquet(f'results/filter2/wijken/{file}')])
        else:
            wijken = gpd.read_parquet(f'results/filter2/wijken/{file}')

# wijken.sample(3)
wijken

,wijkcode,wijknaam,gemeentecode,gemeentenaam,IND_WBI,H2O,OAD,STED,BEV_DICHTH,AANT_INW,...,L1_c_Municipal and governmental services,L1_c_Clubs and schools,"L1_c_gyms, fitness and centers",L1_c_Public fields and pitches,L1_c_Fast food,L1_c_Bars and cafes,L1_c_Stands and takeaway,"L1_c_Bus, tram and metro",L1_c_Public transport,L1_c_Train
1364,WK051802,Wijk 02 Belgisch Park,GM0518,'s-Gravenhage,1.0,NEE,2496.0,2.0,7886.0,8370.0,...,0.0,0.0,0.0,2.0,13.0,8.0,2.0,34.0,0.0,0.0
1365,WK051803,Wijk 03 Westbroekpark en Duttendel,GM0518,'s-Gravenhage,1.0,NEE,1722.0,2.0,1318.0,2260.0,...,1.0,20.0,3.0,0.0,0.0,3.0,0.0,18.0,2.0,0.0
1366,WK051804,Wijk 04 Benoordenhout,GM0518,'s-Gravenhage,1.0,NEE,2191.0,2.0,5623.0,15085.0,...,0.0,36.0,1.0,3.0,6.0,7.0,0.0,65.0,0.0,0.0
1367,WK051805,Wijk 05 Archipelbuurt,GM0518,'s-Gravenhage,1.0,NEE,3432.0,1.0,7227.0,6285.0,...,0.0,3.0,1.0,0.0,0.0,5.0,0.0,17.0,0.0,0.0
1368,WK051806,Wijk 06 Van Stolkpark en Scheveningse Bosjes,GM0518,'s-Gravenhage,1.0,NEE,3543.0,1.0,779.0,845.0,...,0.0,28.0,1.0,0.0,0.0,2.0,0.0,6.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
444,WK019340,Wijk 40 Berkum,GM0193,Zwolle,1.0,NEE,684.0,4.0,283.0,5200.0,...,0.0,28.0,6.0,0.0,6.0,4.0,0.0,70.0,0.0,0.0
445,WK019341,Wijk 41 Marsweteringlanden,GM0193,Zwolle,1.0,NEE,455.0,5.0,71.0,1185.0,...,0.0,13.0,0.0,0.0,2.0,1.0,0.0,12.0,0.0,0.0
446,WK019350,Wijk 50 Schelle,GM0193,Zwolle,1.0,NEE,1618.0,2.0,1828.0,14155.0,...,0.0,23.0,2.0,3.0,2.0,0.0,0.0,36.0,3.0,0.0
447,WK019351,Wijk 51 Ittersum,GM0193,Zwolle,1.0,NEE,1791.0,2.0,4835.0,14625.0,...,0.0,27.0,1.0,3.0,7.0,3.0,0.0,28.0,0.0,0.0


In [24]:
sample = wijken.sample(1)
sample = wijken[wijken['wijknaam'] == 'Rozenknopje']
sample

,wijkcode,wijknaam,gemeentecode,gemeentenaam,IND_WBI,H2O,OAD,STED,BEV_DICHTH,AANT_INW,...,L1_c_Municipal and governmental services,L1_c_Clubs and schools,"L1_c_gyms, fitness and centers",L1_c_Public fields and pitches,L1_c_Fast food,L1_c_Bars and cafes,L1_c_Stands and takeaway,"L1_c_Bus, tram and metro",L1_c_Public transport,L1_c_Train
1822,WK077271,Rozenknopje,GM0772,Eindhoven,1.0,NEE,3626.0,1.0,6306.0,7020.0,...,0.0,3.0,2.0,1.0,5.0,3.0,1.0,20.0,0.0,0.0


In [25]:
agecols = list(wijken.filter(regex="P_.*_JR$").columns)
features = ['BEV_DICHTH', 'L0_altieri', 'L1_altieri']
wijken = wijken[wijken['BEV_DICHTH'] > 0]  # remove rows with 0 population density
# normalize features
wijken[features] = wijken[features].apply(lambda x: (x - x.min()) / (x.max() - x.min()))
wijken[agecols] = wijken[agecols].apply(lambda x: x/100)
sample = wijken.sample(1)
sample = wijken[wijken['gemeentenaam'] == 'Eindhoven']
sample = sample[sample['wijknaam'] == 'Rozenknopje']
s
features = features + agecols
# calculate the distance between the sample and all other wijken, don't use the index
wijken['distance'] = np.linalg.norm(wijken[features] - sample[features].values[0], axis=1)

# sort by distance ascending
similarities = wijken.sort_values('distance').reset_index(drop=True)
# similarities = similarities[~similarities['gemeentenaam'].isin(sample['gemeentenaam'])]
# show the 5 most similar wijken
print(f'5 most similar wijken for {sample.wijknaam.values[0]} - {sample.gemeentenaam.values[0]}:')
similarities.head(10)[['gemeentenaam', 'wijknaam'] + features + ['distance']]


# import plotly.express as px

# fig = px.scatter_3d(wijken, x='BEV_DICHTH', y='L0_altieri', z='L1_altieri', color='distance', hover_data=['gemeentenaam', 'wijknaam'])
# fig.show()

5 most similar wijken for Rozenknopje - Eindhoven:


,gemeentenaam,wijknaam,BEV_DICHTH,L0_altieri,L1_altieri,P_00_14_JR,P_15_24_JR,P_25_44_JR,P_45_64_JR,P_65_EO_JR,distance
0,Eindhoven,Rozenknopje,0.228364,0.912442,0.855112,0.11,0.15,0.33,0.23,0.18,0.000000
1,Eindhoven,De Laak,0.199058,0.908160,0.870689,0.12,0.15,0.36,0.23,0.14,0.060990
2,Groningen,Helpman e.o.,0.164463,0.902205,0.808870,0.13,0.14,0.31,0.23,0.19,0.085595
3,Rotterdam,Charlois,0.229343,0.924824,0.866612,0.17,0.13,0.35,0.23,0.12,0.091030
4,Leeuwarden,Sonnenborgh e.o.,0.172577,0.927866,0.860312,0.14,0.19,0.28,0.23,0.18,0.091527
5,Utrecht,Wijk 07 Zuid,0.219707,0.874628,0.870011,0.16,0.15,0.37,0.22,0.12,0.097606
6,Eindhoven,Erp,0.163775,0.873372,0.823919,0.13,0.15,0.36,0.20,0.15,0.098850
7,Zwolle,Wijk 13 Assendorp,0.158812,0.915434,0.852446,0.14,0.17,0.34,0.22,0.12,0.099768
8,Rotterdam,IJsselmonde,0.190219,0.925475,0.883570,0.18,0.12,0.29,0.24,0.17,0.100174
9,Eindhoven,Oud-Stratum,0.220359,0.875274,0.920025,0.11,0.16,0.38,0.20,0.15,0.100296
